# TTT (Text-to-Text) Client

## 1. Setup

1. Clone the repository.

```bash
git clone https://github.com/kakkoii1337/gai-sdk
```

2. Initialize the SDK.

```bash
gai init
```

3. Download the Model.

```bash
gai pull llamacpp-dolphin
```

4. To test with openai model, create a `.env` file in the root directory of the project and add the following:

```
OPENAI_API_KEY=<REPLACE_WITH_YOUR_OPENAI_API_KEY>
```


## 2. Chat

Local LLM

In [1]:
from gai.ttt.client.ttt_client import TTTClient
client = TTTClient({
    "url":"http://localhost:12031/gen/v1/chat/completions"
})
response=client(messages=[
        {"role":"user","content":"tell me a one sentence story"},
        {"role":"assistant","content":""}
    ],
    stream=False
    )
print(response.extract())

{'type': 'content', 'content': 'The old man, tired of his mundane life, found new purpose when he adopted a stray dog, and together they embarked on a series of thrilling adventures.'}


openai

In [2]:
from gai.ttt.client.ttt_client import TTTClient
client = TTTClient({
    "engine":"openai",
    "model":"gpt-4"
})
response=client(messages=[
        {"role":"user","content":"tell me a one sentence story"},
        {"role":"assistant","content":""}
    ],
    stream=False
    )
print(response.extract())

{'type': 'content', 'content': 'In the midst of a stormy night, a timid, stray kitten found warmth and solace in the old, creaky house which was believed to be haunted.'}


## 3. Stream

Local LLM

In [1]:
from gai.ttt.client.ttt_client import TTTClient
client = TTTClient({
        "url": "http://localhost:12031/gen/v1/chat/completions"
})
response=client(messages=[
    {"role":"user","content":"tell me a one sentence story"},
    {"role":"assistant","content":""}
    ],stream=True)
for chunk in response:
    chunk = chunk.extract()
    print(chunk, end="", flush=True)


An old man sat alone in his house, his only companion the ticking clock on the wall.

openai

In [6]:
from gai.ttt.client.ttt_client import TTTClient
client = TTTClient({
    "engine":"openai",
    "model":"gpt-4"
})
response=client(messages=[
    {"role":"user","content":"tell me a one sentence story"},
    {"role":"assistant","content":""}
],stream=True)
for chunk in response:
    chunk = chunk.extract()
    if type(chunk) is str:
        print(chunk, end="", flush=True)


"In the hushed serenity of a snow-blanketed Christmas morning, grandpa finally received a hand-written letter from his long-lost brother, John, igniting hope and warmth in his old, weary heart."

## 4. Tool Call

Local LLM

In [7]:
messages = [
    {"role":"user","content":"What is the current time in Singapore?"},
    {"role":"assistant","content":""}
]
tool_choice="required"
tools = [
    {
        "type": "function",
        "function": {
            "name": "google",
            "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_query": {
                        "type": "string",
                        "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                    }
                },
                "required": ["search_query"]
            }
        }
    }
]
from gai.ttt.client.ttt_client import TTTClient
client = TTTClient({
        "url": "http://localhost:12031/gen/v1/chat/completions"
})
response=client(messages=messages,stream=False,tool_choice=tool_choice,tools=tools)
print(response.extract())


{'type': 'function', 'name': 'google', 'arguments': '{"search_query": "current time in Singapore"}'}


openai

In [8]:
messages = [
    {"role":"user","content":"What is the current time in Singapore?"},
    {"role":"assistant","content":""}
]
tool_choice="required"
tools = [
    {
        "type": "function",
        "function": {
            "name": "google",
            "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_query": {
                        "type": "string",
                        "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                    }
                },
                "required": ["search_query"]
            }
        }
    }
]
from gai.ttt.client.ttt_client import TTTClient
client = TTTClient({
    "engine":"openai",
    "model":"gpt-4"
})
response=client(messages=messages,stream=False,tool_choice=tool_choice,tools=tools)
print(response.extract())


{'type': 'function', 'name': 'google', 'arguments': '{\n  "search_query": "current time in Singapore"\n}'}


## 5. JSON Schema

local llm

In [2]:
from gai.ttt.client.ttt_client import TTTClient
client = TTTClient({        
        "url": "http://localhost:12031/gen/v1/chat/completions"
})
# Define Schema
from pydantic import BaseModel
class Book(BaseModel):
    title: str
    summary: str
    author: str
    published_year: int

response=client(
    messages=[
        {"role":"user","content":"""Foundation is a science fiction novel by American writer 
            Isaac Asimov. It is the first published in his Foundation Trilogy (later 
            expanded into the Foundation series). Foundation is a cycle of five 
            interrelated short stories, first published as a single book by Gnome Press 
            in 1951. Collectively they tell the early story of the Foundation, 
            an institute founded by psychohistorian Hari Seldon to preserve the best 
            of galactic civilization after the collapse of the Galactic Empire.
            """},
        {"role":"assistant","content":""}
        ],
    response_format=Book,
    tool_choice="none",
    stream=False
    )
print(response.extract())

{'type': 'content', 'content': '{ "title": "Foundation", "summary": "Foundation is a science fiction novel by American writer Isaac Asimov. It is the first published in his Foundation Trilogy (later expanded into the Foundation series). Foundation is a cycle of five interrelated short stories, first published as a single book by Gnome Press in 1951. Collectively they tell the early story of the Foundation, an institute founded by psychohistorian Hari Seldon to preserve the best of galactic civilization after the collapse of the Galactic Empire." ,"author": "Isaac Asimov","published_year": 1951}'}


openai

In [1]:
from gai.ttt.client.ttt_client import TTTClient
client = TTTClient({
    "engine":"openai",
    "model":"gpt-4o"
})
# Define Schema
from pydantic import BaseModel
class Book(BaseModel):
    title: str
    summary: str
    author: str
    published_year: int

response=client(
    messages=[
        {"role":"user","content":"""Foundation is a science fiction novel by American writer 
            Isaac Asimov. It is the first published in his Foundation Trilogy (later 
            expanded into the Foundation series). Foundation is a cycle of five 
            interrelated short stories, first published as a single book by Gnome Press 
            in 1951. Collectively they tell the early story of the Foundation, 
            an institute founded by psychohistorian Hari Seldon to preserve the best 
            of galactic civilization after the collapse of the Galactic Empire.
            """},
        {"role":"assistant","content":""}
        ],
    response_format=Book,
    tool_choice=None,
    stream=False
    )
print(response.extract())

{'type': 'content', 'content': '{"title":"Foundation","summary":"\\"Foundation\\" is a seminal science fiction novel that weaves together a series of interconnected stories set in a future where the Galactic Empire is on the brink of collapse. The narrative revolves around Hari Seldon, a visionary psychohistorian, who predicts the fall of the empire and creates the Foundation — a consortium of scientists and scholars — to safeguard knowledge and the remnants of civilization. The novel explores themes of decay, the rise of new powers, and the struggle to maintain knowledge in tumultuous times.","author":"Isaac Asimov","published_year":1951}'}
